In [ ]:
import requests
import pandas as pd

Extraindo os dados proveniente do data sus

In [ ]:
YEAR = 2025
url = (
        "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/"
        "SRAG/2025/INFLUD25-10-11-2025.csv"
)

response = requests.get(url)
response.raise_for_status()

df = pd.read_csv(url, sep=';', encoding='latin1')
df.head()

/tmp/ipython-input-257838647.py:10: DtypeWarning: Columns (14,92,94,185,186,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=';', encoding='latin1')


,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,31735451969802,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,31735516593521,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,31735544021940,2024-12-29,1,2024-12-29,1,SP,GVE XXIX SAO JOSE DO RIO PRETO,1354.0,SAO JOSE DO RIO PRETO,354980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31735559043222,2024-12-30,1,2024-12-29,1,AC,REGIONAL DO BAIXO ACRE,1938.0,RIO BRANCO,120040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,31735560267779,2024-12-30,1,2024-12-29,1,PB,I NRS JOAO PESSOA,1377.0,JOAO PESSOA,250750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


Removendo colunas irrelevantes para o problema, retirando dados nulos e substiuindo os valores nan por 0

In [ ]:
columns_important = [
        'DT_NOTIFIC', 'AVE_SUINO', 'FEBRE',
        'DISPNEIA', 'DESC_RESP','FATOR_RISC',
        'VACINA', 'ANTIVIRAL', 'UTI','EVOLUCAO','SURTO_SG'
]

df_raw_important = df[columns_important]
df_treated_nan = df_raw_important.fillna("0")
df_treated = df_treated_nan.drop_duplicates()

Substiuindo os valores das colunas relevantes por um contéudo que seja de fácil compreensão para a LLM

In [ ]:
output_path = "data-gold"

map_yes_no = {
        0: "Campo vazio", 1: "Sim", 2: "Não", 9: "Ignorado"
}
map_birds_pigs = {
        0: "Campo vazio", 1: "Sim, aves e/ou suínos", 2: "Não, nenhum",
        3: "Sim, outros", 9: "Ignorado"
    }
map_evolucao = {
        0: "Campo vazio", 1: "Cura", 2: "Óbito", 3: "Óbito por outras causa", 9: "Ignorado"
    }

columns_yes_no = [
        'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 
        'FATOR_RISC', 'VACINA', 'ANTIVIRAL', 'UTI', 'SURTO_SG'
    ]


df_treated[columns_yes_no] = df_treated[columns_yes_no].apply(lambda col: col.map(map_yes_no))

df_treated['AVE_SUINO'] = df_treated['AVE_SUINO'].map(map_birds_pigs)
df_treated['EVOLUCAO'] = df_treated['EVOLUCAO'].map(map_evolucao)

df_treated['DT_NOTIFIC'] = pd.to_datetime(df_treated['DT_NOTIFIC'], errors='coerce') \
        .dt.strftime('%Y-%m-%d')

df_treated.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,31735451969802,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,31735516593521,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,31735544021940,2024-12-29,1,2024-12-29,1,SP,GVE XXIX SAO JOSE DO RIO PRETO,1354.0,SAO JOSE DO RIO PRETO,354980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31735559043222,2024-12-30,1,2024-12-29,1,AC,REGIONAL DO BAIXO ACRE,1938.0,RIO BRANCO,120040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,31735560267779,2024-12-30,1,2024-12-29,1,PB,I NRS JOAO PESSOA,1377.0,JOAO PESSOA,250750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
